# Enhanced CPS Dataset Analysis

This document describes the Enhanced CPS dataset implementation, including the ASEC Undocumented Algorithm for SSN card type assignment and validation testing for policy reform impacts such as Child Tax Credit (CTC) analysis.

## Overview

The Enhanced CPS dataset integrates multiple sophisticated algorithms to create a comprehensive microsimulation dataset that accurately represents the US population, including undocumented immigrants and their policy interactions.

# Part I: ASEC Undocumented Algorithm

The ASEC Undocumented Algorithm implements a process of elimination approach to identify individuals who are likely undocumented immigrants by systematically removing people who have clear indicators of legal immigration status.

**Research Paper**: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4662801

## Algorithm Overview

The algorithm assigns SSN card type codes:
- **Code 0**: `"NONE"` - Likely undocumented immigrants
- **Code 1**: `"CITIZEN"` - US citizens (born or naturalized)  
- **Code 2**: `"NON_CITIZEN_VALID_EAD"` - Non-citizens with work/study authorization
- **Code 3**: `"OTHER_NON_CITIZEN"` - Non-citizens with indicators of legal status

## Implementation Details

The algorithm is implemented in the `add_ssn_card_type()` function in `policyengine_us_data/datasets/cps/cps.py:848-1305`.

### Function Parameters
- `undocumented_target`: 13 million (default target for total undocumented population)
- `undocumented_workers_target`: 8.3 million (target for undocumented workers)
- `undocumented_students_target`: ~399k (21% of 1.9M total undocumented students)

## Algorithm Steps

### Step 0: Initialization
- All persons start with SSN card type code 0 (`ssn_card_type = np.full(len(person), 0)`)
- Prints initial Code 0 population count

### Step 1: Citizen Classification
Citizens are identified and moved to code 1 based on citizenship status:
- **Condition**: `PRCITSHP` values 1, 2, 3, or 4 (all citizen types)
- **Result**: Citizens assigned to code 1, non-citizens (`PRCITSHP == 5`) remain for further processing

### Step 2: ASEC Undocumented Algorithm Conditions
The algorithm applies 14 specific conditions to identify non-citizens with indicators of legal status. Only individuals not already classified as citizens (code 1) or authorized workers/students (code 2) are evaluated (`potentially_undocumented` mask).

#### Condition 1: Pre-1982 Arrivals (IRCA Amnesty Eligible)
- **Variable**: `PEINUSYR` codes 1-7
- **Logic**: Immigrants who arrived before 1982 were eligible for IRCA amnesty
- **Codes**: 01=Before 1950, 02=1950-1959, 03=1960-1964, 04=1965-1969, 05=1970-1974, 06=1975-1979, 07=1980-1981

#### Condition 2: Eligible Naturalized Citizens
- **Variables**: `PRCITSHP == 4` (naturalized citizen), `A_AGE >= 18`, `PEINUSYR` for residency
- **Logic**: Naturalized citizens meeting residency requirements:
  - 5+ years in US (codes 8-26: 1982-2019), OR
  - 3+ years in US + married to citizen (codes 8-27: 1982-2021)
- **Marriage check**: `A_MARITL` in [1,2] AND `A_SPOUSE > 0`

#### Condition 3: Medicare Recipients
- **Variable**: `MCARE == 1`
- **Logic**: Medicare eligibility indicates legal status

#### Condition 4: Federal Retirement Benefits
- **Variables**: `PEN_SC1 == 3` OR `PEN_SC2 == 3`
- **Logic**: Federal government pension recipients have legal work history

#### Condition 5: Social Security Disability
- **Variables**: `RESNSS1 == 2` OR `RESNSS2 == 2`
- **Logic**: Social Security disability benefits indicate legal status (code 2 = disabled adult or child)

#### Condition 6: Indian Health Service Coverage
- **Variable**: `IHSFLG == 1`
- **Logic**: IHS coverage indicates legal status or tribal membership

#### Condition 7: Medicaid Recipients
- **Variable**: `CAID == 1`
- **Logic**: Medicaid eligibility generally requires legal status (simplified implementation without state-specific adjustments)

#### Condition 8: CHAMPVA Recipients
- **Variable**: `CHAMPVA == 1`
- **Logic**: CHAMPVA (Veterans Affairs health coverage) indicates military family connection

#### Condition 9: Military Health Insurance
- **Variable**: `MIL == 1`
- **Logic**: TRICARE/military health insurance indicates military service or family connection

#### Condition 10: Government Employees
- **Variables**: `PEIO1COW` codes 1-3 OR `A_MJOCC == 11`
- **Logic**: Government employment requires legal work authorization
- **Codes**: 1=Federal government, 2=State government, 3=Local government, A_MJOCC 11=Military occupation

#### Condition 11: Social Security Recipients
- **Variable**: `SS_YN == 1`
- **Logic**: Social Security benefits indicate legal status and work history

#### Condition 12: Housing Assistance
- **Variable**: `SPM_CAPHOUSESUB > 0` (mapped from SPM unit data)
- **Logic**: Housing assistance programs generally require legal status
- **Implementation**: Uses SPM unit mapping: `spm_housing_map = dict(zip(smp_unit.SPM_ID, spm_unit.SPM_CAPHOUSESUB))`

#### Condition 13: Veterans/Military Personnel
- **Variables**: `PEAFEVER == 1` OR `A_MJOCC == 11`
- **Logic**: Military service or veteran status indicates legal status

#### Condition 14: SSI Recipients
- **Variable**: `SSI_YN == 1`
- **Logic**: SSI eligibility generally requires legal status (simplified implementation)

### Step 3: Target-Driven EAD Assignment for Workers
- **Target**: 8.3 million undocumented workers (from Pew Research)
- **Eligibility**: Non-citizens not in code 3 with earnings (`WSAL_VAL > 0` OR `SEMP_VAL > 0`)
- **Process**: Uses `select_random_subset_to_target()` function with random seed 0
- **Logic**: Calculates how many workers need EAD status to hit the target, then randomly selects from eligible pool
- **Result**: Selected workers moved from code 0 to code 2

### Step 4: Target-Driven EAD Assignment for Students  
- **Target**: ~399k undocumented students (21% of 1.9M total, from Higher Ed Immigration Portal)
- **Eligibility**: Non-citizens not in code 3 currently in college (`A_HSCOL == 2`)
- **Process**: Uses `select_random_subset_to_target()` function with random seed 1
- **Result**: Selected students moved from code 0 to code 2

### Step 5: Family Correlation Adjustment
- **Logic**: If any household member has code 0 (undocumented), all other eligible household members are also assigned code 0
- **Scope**: Only affects people with codes 0 or 3 (not citizens or EAD holders)
- **Process**: 
  1. Iterate through each unique household ID
  2. Check if household has any code 0 members
  3. If yes, change all code 3 members in that household to code 0
- **Implementation**: 100% correlation within households (not probabilistic)

### Step 6: Final Population Targeting
- **Target**: 13 million total undocumented immigrants
- **Process**: Uses `select_random_subset_to_target()` function with random seed 42
- **Logic**: If current undocumented population exceeds target, randomly move some code 0 individuals to code 3
- **Eligibility**: Remaining code 0 non-citizens only

## Population Data and Tracking

The `add_ssn_card_type()` function automatically generates a CSV file (`asec_population_log.csv`) that contains all population numbers from each step of the algorithm. This file is saved to the `docs/` directory and contains the following columns:

- **step**: The algorithm step or condition name
- **description**: Description of what the population number represents
- **population**: The weighted population count

In [ ]:
import pandas as pd
import os

# Read the population log CSV file
csv_path = "asec_population_log.csv"
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print("✅ Population data loaded from CSV file\n")
    print(df.to_string(index=False))
else:
    print("❌ CSV file not found. Run cps.py to generate population data.")
    df = pd.DataFrame()  # Empty dataframe for subsequent cells

### Actual Population Numbers (from latest run)

In [ ]:
if not df.empty:
    def get_population(step, description):
        """Helper function to get population for a specific step and description"""
        result = df[(df['step'] == step) & (df['description'] == description)]
        if not result.empty:
            return f"{result.iloc[0]['population']:,.0f}"
        return "Not found"
    
    print("## Algorithm Step Results\n")
    
    print("### Initialization")
    print(f"- **Step 0 - Initial**: Code 0 people = {get_population('Step 0 - Initial', 'Code 0 people')}")
    print()
    
    print("### Citizen Classification")
    print(f"- **Step 1 - Citizens**: Moved to Code 1 = {get_population('Step 1 - Citizens', 'Moved to Code 1')}")
    print()
    
    print("### ASEC Conditions Analysis")
    print(f"- **ASEC Conditions**: Current Code 0 people = {get_population('ASEC Conditions', 'Current Code 0 people')}")
    print(f"- **After conditions**: Code 0 people = {get_population('After conditions', 'Code 0 people')}")
    print()
    
    print("### Target Information")
    print(f"- **Before adjustment**: Undocumented workers = {get_population('Before adjustment', 'Undocumented workers')}")
    print(f"- **Target**: Undocumented workers target = {get_population('Target', 'Undocumented workers target')}")
    print(f"- **Before adjustment**: Undocumented students = {get_population('Before adjustment', 'Undocumented students')}")
    print(f"- **Target**: Undocumented students target = {get_population('Target', 'Undocumented students target')}")
    print()
    
    print("### EAD Assignment")
    print(f"- **Step 3 - EAD workers**: Moved from Code 0 to Code 2 = {get_population('Step 3 - EAD workers', 'Moved from Code 0 to Code 2')}")
    print(f"- **Step 4 - EAD students**: Moved from Code 0 to Code 2 = {get_population('Step 4 - EAD students', 'Moved from Code 0 to Code 2')}")
    print(f"- **After EAD assignment**: Code 0 people = {get_population('After EAD assignment', 'Code 0 people')}")
    print()
    
    print("### Family Correlation")
    print(f"- **Step 5 - Family correlation**: Changed from Code 3 to Code 0 = {get_population('Step 5 - Family correlation', 'Changed from Code 3 to Code 0')}")
    print(f"- **After family correlation**: Code 0 people = {get_population('After family correlation', 'Code 0 people')}")
    print()
    
    print("### Target Refinement")
    print(f"- **Step 6 - Target refinement**: Moved from Code 0 to Code 3 = {get_population('Step 6 - Target refinement', 'Moved from Code 0 to Code 3')}")
    print(f"- **After target refinement**: Code 0 people = {get_population('After target refinement', 'Code 0 people')}")
    print()
    
    print("### Final Results")
    print(f"- **Final**: Code 0 (NONE) = {get_population('Final', 'Code 0 (NONE)')}")
    print(f"- **Final**: Code 1 (CITIZEN) = {get_population('Final', 'Code 1 (CITIZEN)')}")
    print(f"- **Final**: Code 2 (NON_CITIZEN_VALID_EAD) = {get_population('Final', 'Code 2 (NON_CITIZEN_VALID_EAD)')}")
    print(f"- **Final**: Code 3 (OTHER_NON_CITIZEN) = {get_population('Final', 'Code 3 (OTHER_NON_CITIZEN)')}")
    print(f"- **Final**: Total undocumented (Code 0) = {get_population('Final', 'Total undocumented (Code 0)')}")
    print(f"- **Final**: Undocumented target = {get_population('Final', 'Undocumented target')}")
else:
    print("*Run cps.py to generate population data*")

## Helper Functions

### `select_random_subset_to_target()`
A sophisticated targeting function that handles both population reduction and increase scenarios:
- **Population reduction**: Uses new random number generator (`np.random.default_rng`) for refinement steps
- **Population increase**: Uses legacy `np.random` for EAD assignment compatibility
- **Weighting**: Accounts for household weights in selection probability
- **Bounds checking**: Caps selection percentage at 100%

## SSN Card Type Calibration in Loss Function

The ASEC undocumented algorithm is integrated into PolicyEngine's calibration system through the loss function in `policyengine_us_data/utils/loss.py`. This ensures that the simulated undocumented population matches external estimates.

### Calibration Implementation

The calibration system targets the `"NONE"` SSN card type (undocumented immigrants) using the following approach:

```python
# SSN Card Type calibration
for card_type_str in ["NONE"]:  # SSN card types as strings
    ssn_type_mask = sim.calculate("ssn_card_type").values == card_type_str
    
    # Overall count by SSN card type
    label = f"ssa/ssn_card_type_{card_type_str.lower()}_count"
    loss_matrix[label] = sim.map_result(
        ssn_type_mask, "person", "household"
    )
```

### Target Population by Year

The calibration uses year-specific targets based on authoritative sources:

```python
undocumented_targets = {
    2022: 11.0e6,  # Official DHS Office of Homeland Security Statistics estimate for 1 Jan 2022
    # https://ohss.dhs.gov/sites/default/files/2024-06/2024_0418_ohss_estimates-of-the-unauthorized-immigrant-population-residing-in-the-united-states-january-2018%25E2%2580%2593january-2022.pdf
    2023: 12.2e6,  # Center for Migration Studies ACS-based residual estimate (published May 2025)
    # https://cmsny.org/publications/the-undocumented-population-in-the-united-states-increased-to-12-million-in-2023/
    2024: 13.0e6,  # Reuters synthesis of experts ahead of 2025 change ("~13-14 million") - central value
    # https://www.reuters.com/data/who-are-immigrants-who-could-be-targeted-trumps-mass-deportation-plans-2024-12-18/
    2025: 13.0e6,  # Same midpoint carried forward - CBP data show 95% drop in border apprehensions
}
```

### Logic for Target Selection

```python
if time_period <= 2022:
    target_count = 11.0e6  # Use 2022 value for earlier years
elif time_period >= 2025:
    target_count = 13.0e6  # Use 2025 value for later years
else:
    target_count = undocumented_targets[time_period]
```

### Data Sources

1. **2022 (11.0M)**: Official DHS Office of Homeland Security Statistics estimate for January 1, 2022
   - [Source: DHS Report](https://ohss.dhs.gov/sites/default/files/2024-06/2024_0418_ohss_estimates-of-the-unauthorized-immigrant-population-residing-in-the-united-states-january-2018%25E2%2580%2593january-2022.pdf)

2. **2023 (12.2M)**: Center for Migration Studies ACS-based residual estimate
   - [Source: CMS Report](https://cmsny.org/publications/the-undocumented-population-in-the-united-states-increased-to-12-million-in-2023/)

3. **2024-2025 (13.0M)**: Reuters synthesis of expert estimates
   - [Source: Reuters Analysis](https://www.reuters.com/data/who-are-immigrants-who-could-be-targeted-trumps-mass-deportation-plans-2024-12-18/)

### Integration with Algorithm

The calibration target (13.0M for 2024) directly corresponds to the `undocumented_target` parameter in the `add_ssn_card_type()` function, ensuring consistency between the algorithm implementation and the broader PolicyEngine calibration framework.

This calibration ensures that PolicyEngine microsimulations produce realistic estimates of undocumented population sizes that align with the best available research and government statistics.

# Part II: CTC Reform Child Recipient Difference Analysis

The Enhanced CPS dataset includes validation testing for policy reform impacts, including the Child Tax Credit (CTC) reform analysis. This test validates the dataset's ability to model policy changes affecting families, particularly those with different immigration statuses.

## Overview

The `test_ctc_reform_child_recipient_difference()` function compares baseline and reform scenarios to measure the impact on child CTC recipients. This analysis helps validate the Enhanced CPS dataset's policy modeling capabilities.

**Note**: This test is used for validation purposes only and is no longer actively targeted in loss matrix calibration due to uncertainty around assumptions from hearing comments.

## Test Implementation

The test implements the following logic:

1. **Define CTC Reform**: Creates a reform that enables the reconciliation CTC provision
2. **Create Simulations**: Runs both baseline and reform microsimulations
3. **Calculate Recipients**: Identifies child CTC recipients in both scenarios
4. **Measure Difference**: Computes the change in child recipients between scenarios
5. **Validate Against Target**: Compares results to a target of 2 million with ±400% tolerance

### Key Parameters

- **Target Count**: 2,000,000 child recipient difference
- **Tolerance**: ±400% error allowed
- **Reform Period**: 2025-01-01 to 2100-12-31
- **Recipient Criteria**: Children with `ctc_individual_maximum > 0` AND `ctc_value > 0`

In [ ]:
def test_ctc_reform_child_recipient_difference():
    """
    Test CTC reform impact for validation purposes only.
    Note: This is no longer actively targeted in loss matrix calibration
    due to uncertainty around assumptions from hearing comments.
    """
    from policyengine_us_data.datasets.cps import EnhancedCPS_2024
    from policyengine_us import Microsimulation
    from policyengine_core.reforms import Reform

    TARGET_COUNT = 2e6
    TOLERANCE = 4  # Allow ±400% error

    # Define the CTC reform
    ctc_reform = Reform.from_dict(
        {
            "gov.contrib.reconciliation.ctc.in_effect": {
                "2025-01-01.2100-12-31": True
            }
        },
        country_id="us",
    )

    # Create baseline and reform simulations
    baseline_sim = Microsimulation(dataset=EnhancedCPS_2024)
    reform_sim = Microsimulation(dataset=EnhancedCPS_2024, reform=ctc_reform)

    # Calculate baseline CTC recipients (children with ctc_individual_maximum > 0 and ctc_value > 0)
    baseline_is_child = baseline_sim.calculate("is_child")
    baseline_ctc_individual_maximum = baseline_sim.calculate(
        "ctc_individual_maximum"
    )
    baseline_ctc_value = baseline_sim.calculate("ctc_value", map_to="person")
    baseline_child_ctc_recipients = (
        baseline_is_child
        * (baseline_ctc_individual_maximum > 0)
        * (baseline_ctc_value > 0)
    ).sum()

    # Calculate reform CTC recipients (children with ctc_individual_maximum > 0 and ctc_value > 0)
    reform_is_child = reform_sim.calculate("is_child")
    reform_ctc_individual_maximum = reform_sim.calculate(
        "ctc_individual_maximum"
    )
    reform_ctc_value = reform_sim.calculate("ctc_value", map_to="person")
    reform_child_ctc_recipients = (
        reform_is_child
        * (reform_ctc_individual_maximum > 0)
        * (reform_ctc_value > 0)
    ).sum()

    # Calculate the difference (baseline - reform child CTC recipients)
    ctc_recipient_difference = (
        baseline_child_ctc_recipients - reform_child_ctc_recipients
    )

    pct_error = abs((ctc_recipient_difference - TARGET_COUNT) / TARGET_COUNT)

    print(
        f"CTC reform child recipient difference: {ctc_recipient_difference:.0f}, target: {TARGET_COUNT:.0f}, error: {pct_error:.2%}"
    )
    print(
        "Note: CTC targeting removed from calibration - this is validation only"
    )
    
    return {
        'baseline_child_ctc_recipients': baseline_child_ctc_recipients,
        'reform_child_ctc_recipients': reform_child_ctc_recipients,
        'ctc_recipient_difference': ctc_recipient_difference,
        'target_count': TARGET_COUNT,
        'pct_error': pct_error,
        'tolerance': TOLERANCE
    }

## Run CTC Analysis

Execute the CTC reform analysis and display detailed results:

In [ ]:
# Run the test and display results
try:
    print("Running CTC Reform Analysis...")
    print("This may take a few minutes to load the dataset and run simulations.\n")
    
    results = test_ctc_reform_child_recipient_difference()
    
    print("\n" + "="*60)
    print("CTC REFORM ANALYSIS RESULTS")
    print("="*60)
    print(f"Baseline child CTC recipients: {results['baseline_child_ctc_recipients']:,.0f}")
    print(f"Reform child CTC recipients: {results['reform_child_ctc_recipients']:,.0f}")
    print(f"Difference (baseline - reform): {results['ctc_recipient_difference']:,.0f}")
    print(f"Target difference: {results['target_count']:,.0f}")
    print(f"Percent error: {results['pct_error']:.2%}")
    print(f"Tolerance: ±{results['tolerance']*100:.0f}%")
    
    # Determine if test passes
    if results['pct_error'] <= results['tolerance']:
        print("\n✅ TEST PASSED: Error within tolerance")
    else:
        print("\n❌ TEST FAILED: Error exceeds tolerance")
    
    print("\nThis test validates the dataset's ability to model CTC policy impacts.")
    print("Note: This is validation only - not used for active calibration.")
    
except Exception as e:
    print(f"❌ Test failed to run: {e}")
    print("\nPossible causes:")
    print("- Enhanced CPS dataset not generated yet")
    print("- Missing PolicyEngine dependencies")
    print("- Insufficient memory for microsimulation")
    print("\nTo resolve, ensure the Enhanced CPS dataset is available and all dependencies are installed.")

## Analysis Interpretation

### What the Numbers Mean

- **Baseline Recipients**: Number of children receiving CTC under current law
- **Reform Recipients**: Number of children receiving CTC under the reform scenario
- **Difference**: How many fewer children receive CTC under baseline vs reform
- **Target**: Expected difference based on policy analysis (2 million)

### Policy Context

The CTC reform being tested (`gov.contrib.reconciliation.ctc.in_effect`) represents changes to Child Tax Credit eligibility and amounts. The difference in recipients between baseline and reform scenarios helps validate that the Enhanced CPS dataset accurately models:

1. **Immigration Status Effects**: How CTC changes affect families with different SSN card types
2. **Income Distribution**: Whether the dataset captures realistic income patterns
3. **Family Composition**: Accurate representation of household structures

### Validation Purpose

This test serves as a validation check rather than an active calibration target because:
- Policy assumptions involve uncertainty from legislative hearing comments
- The 400% tolerance reflects this uncertainty
- Results help confirm overall dataset quality without over-constraining the model

## Usage

When you run the CPS data generation process, the function will:
1. Execute all algorithm steps
2. Print population numbers to console
3. Save detailed population log to `docs/asec_population_log.csv` (replacing previous file)
4. Display the CSV file path upon completion
5. Automatically update this documentation with the latest numbers

The population numbers in this notebook are dynamically loaded from the CSV file and will always reflect the most recent algorithm run.

To run the complete analysis including both ASEC algorithm and CTC validation:
1. Ensure the Enhanced CPS 2024 dataset is generated
2. Install required dependencies (`policyengine-us`, `policyengine-core`)
3. Execute the notebook cells in order
4. Review results from both algorithm execution and validation testing